In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
datos = pd.read_excel('waves_WW3_15.75S_75W.xlsx', usecols=['Hs (m)', 'Tp (s)', 'Dir'])

Para orientar las direcciones conforme a la costa, restamos 208°

In [3]:
datos['Dir'] = datos['Dir'] - 208

In [4]:
# Habiendo restado -208° vemos como se modifican los valores a fin de poder ver que intervalos van a quedar mejor
print(datos['Dir'].min(), datos['Dir'].max())
print(datos['Hs (m)'].min(), datos['Hs (m)'].max())

-57.91999999999999 85.12
0.78 4.79


In [5]:
bins_H = pd.interval_range(start=0, end=5, freq=1,closed='left')
bins_D = pd.interval_range(start=-180, end=180, freq=36,closed='left')

In [6]:
datos['Dir_bins']=pd.cut(datos['Dir'],bins_D)
datos['Hs_bins']=pd.cut(datos['Hs (m)'],bins_H)

Con esto podemos obtener el total de casos de olas encasilladas en dadas direcciones (Dir) y alturas significativas (Hs)

In [7]:
tabla = pd.crosstab(datos.Hs_bins,datos.Dir_bins)
tabla = tabla.rename_axis('Dir (°)', axis=1)
tabla = tabla.rename_axis('Hs (m)', axis=0)
tabla

Dir (°),"[-72, -36)","[-36, 0)","[0, 36)","[36, 72)","[72, 108)"
Hs (m),,,,,
"[0, 1)",1,9,11,4,0
"[1, 2)",11,5204,4654,85,137
"[2, 3)",14,7085,4542,0,0
"[3, 4)",0,1237,513,0,0
"[4, 5)",0,45,49,0,0


Sin embargo, se trata de todos los casos de valores de período pico. Pero dado que en el enunciado nos piden separar por al menos 3 intervalos, podemos decidir en separar por intervalos intercuartiles 

In [8]:
quantiles = datos['Tp (s)'].quantile([.25, .75]).tolist()

# quantiles[0] = 12.06, quantiles[1] = 14.51

datos_1 = datos[datos['Tp (s)'] <= quantiles[0]]
datos_2 = datos[(datos['Tp (s)'] > quantiles[0]) & (datos['Tp (s)'] <= quantiles[1])]
datos_3 = datos[datos['Tp (s)'] > quantiles[1]]

len(datos) == len(datos_1) + len(datos_2) + len(datos_3)

True

In [9]:
print(datos_1['Tp (s)'].mean(), datos_2['Tp (s)'].mean(), datos_3['Tp (s)'].mean())

11.088253834485084 13.312438300398611 15.836382508082355


In [10]:
# Lista de datos y etiquetas
datos_list = [datos_1, datos_2, datos_3]
etiquetas = ['datos_1', 'datos_2', 'datos_3']

# Diccionario para almacenar las tablas
tablas = {}

# Procesar cada conjunto de datos
for datos_i, etiqueta in zip(datos_list, etiquetas):
    tabla = pd.crosstab(datos_i['Hs_bins'], datos_i['Dir_bins'], margins=True, margins_name='Suma Total', normalize='all')
    tabla = tabla.rename_axis('Dir (°)', axis=1)
    tabla = tabla.rename_axis('Hs (m)', axis=0)
    tablas[etiqueta] = tabla
    tabla.to_csv(f'valores_rango_{etiqueta}.csv', index= False)

# Mostrar las tablas
for etiqueta, tabla in tablas.items():
    print(f"Tabla para {etiqueta}:")
    print(tabla)
    print("\n")

Tabla para datos_1:
Dir (°)     [-72, -36)  [-36, 0)   [0, 36)  Suma Total
Hs (m)                                                
[0, 1)        0.000169  0.001348  0.000506    0.002023
[1, 2)        0.001854  0.478341  0.223833    0.704028
[2, 3)        0.002360  0.247767  0.038261    0.288387
[3, 4)        0.000000  0.005562  0.000000    0.005562
Suma Total    0.004382  0.733019  0.262599    1.000000


Tabla para datos_2:
Dir (°)     [-36, 0)   [0, 36)  [36, 72)  [72, 108)  Suma Total
Hs (m)                                                         
[0, 1)      0.000085  0.000509  0.000170   0.000000    0.000763
[1, 2)      0.166907  0.205835  0.001527   0.000763    0.375032
[2, 3)      0.341701  0.219150  0.000000   0.000000    0.560851
[3, 4)      0.045374  0.017725  0.000000   0.000000    0.063099
[4, 5)      0.000254  0.000000  0.000000   0.000000    0.000254
Suma Total  0.554321  0.443219  0.001696   0.000763    1.000000


Tabla para datos_3:
Dir (°)     [-36, 0)   [0, 36)  [36, 72

In [11]:
porcentajes = [len(datos_i) / len(datos) for datos_i in datos_list]
porcentajes

[0.25138765306554806, 0.49959747468327614, 0.2490148722511758]

In [16]:
# Lista de datos y etiquetas
datos_list = [datos_1, datos_2, datos_3]
etiquetas = ['datos_1', 'datos_2', 'datos_3']

# Diccionario para almacenar las tablas
tablas = {}

# Procesar cada conjunto de datos
for datos_i, porcentajes_i, etiqueta in zip(datos_list, porcentajes, etiquetas):
    tabla = pd.crosstab(datos_i['Hs_bins'], datos_i['Dir_bins'], margins=True, margins_name='Suma Total', normalize='all') * porcentajes_i * 100
    tabla = tabla.round(4)  # Redondear a 4 decimales
    tabla = tabla.rename_axis('Dir (°)', axis=1)
    tabla = tabla.rename_axis('Hs (m)', axis=0)
    tablas[etiqueta] = tabla
    tabla.to_csv(f"valores_rango_{etiqueta}.csv")

# Mostrar las tablas
for etiqueta, tabla in tablas.items():
    print(f"Tabla para {etiqueta}:")
    print(tabla)
    print("\n")

Tabla para datos_1:
Dir (°)     [-72, -36)  [-36, 0)  [0, 36)  Suma Total
Hs (m)                                               
[0, 1)          0.0042    0.0339   0.0127      0.0508
[1, 2)          0.0466   12.0249   5.6269     17.6984
[2, 3)          0.0593    6.2285   0.9618      7.2497
[3, 4)          0.0000    0.1398   0.0000      0.1398
Suma Total      0.1102   18.4272   6.6014     25.1388


Tabla para datos_2:
Dir (°)     [-36, 0)  [0, 36)  [36, 72)  [72, 108)  Suma Total
Hs (m)                                                        
[0, 1)        0.0042   0.0254    0.0085     0.0000      0.0381
[1, 2)        8.3386  10.2835    0.0763     0.0381     18.7365
[2, 3)       17.0713  10.9487    0.0000     0.0000     28.0200
[3, 4)        2.2669   0.8856    0.0000     0.0000      3.1524
[4, 5)        0.0127   0.0000    0.0000     0.0000      0.0127
Suma Total   27.6937  22.1431    0.0847     0.0381     49.9597


Tabla para datos_3:
Dir (°)     [-36, 0)  [0, 36)  [36, 72)  [72, 108)  Su

In [13]:
# Lista de valores a añadir
valores = [
    {'T': 11.08, 'Fi0':-54 , 'Hs':0.5, 'Occ':0.05084},
    {'T': 11.08, 'Fi0':-54 , 'Hs':1.5, 'Occ':17.6984},
    {'T': 11.08, 'Fi0':-54,  'Hs':2.5, 'Occ':7.24969},
    {'T': 11.08, 'Fi0':-54 , 'Hs':3.5, 'Occ':0.13982},
    {'T': 11.08, 'Fi0':-18 , 'Hs':0.5, 'Occ':0.05084},
    {'T': 11.08, 'Fi0':-18 , 'Hs':1.5, 'Occ':17.6984},
    {'T': 11.08, 'Fi0':-18 , 'Hs':2.5, 'Occ':7.24969},
    {'T': 11.08, 'Fi0':-18 , 'Hs':3.5, 'Occ':0.13982},
    {'T': 11.08, 'Fi0':18  , 'Hs':0.5, 'Occ':0.05084},
    {'T': 11.08, 'Fi0':18  , 'Hs':1.5, 'Occ':17.6984},
    {'T': 11.08, 'Fi0':18  , 'Hs':2.5, 'Occ':7.24969},
    {'T': 11.08, 'Fi0':18  , 'Hs':3.5, 'Occ':0.13982},
    
    {'T': 13.31, 'Fi0':-18 , 'Hs':0.5, 'Occ': 0.038134},
    {'T': 13.31, 'Fi0':-18 , 'Hs':1.5, 'Occ': 18.73649},
    {'T': 13.31, 'Fi0':-18 , 'Hs':2.5, 'Occ': 28.01999},
    {'T': 13.31, 'Fi0':-18 , 'Hs':3.5, 'Occ': 3.152409},
    {'T': 13.31, 'Fi0':-18 , 'Hs':4.5, 'Occ': 0.012711},
    {'T': 13.31, 'Fi0':18  , 'Hs':0.5, 'Occ': 0.038134},
    {'T': 13.31, 'Fi0':18  , 'Hs':1.5, 'Occ': 18.73649},
    {'T': 13.31, 'Fi0':18  , 'Hs':2.5, 'Occ': 28.01999},
    {'T': 13.31, 'Fi0':18  , 'Hs':3.5, 'Occ': 3.152409},
    {'T': 13.31, 'Fi0':18  , 'Hs':4.5, 'Occ': 0.012711},
    {'T': 13.31, 'Fi0':54 , 'Hs':0.5, 'Occ': 0.038134 },
    {'T': 13.31, 'Fi0':54 , 'Hs':1.5, 'Occ': 18.73649 },
    {'T': 13.31, 'Fi0':54 , 'Hs':2.5, 'Occ': 28.01999 },
    {'T': 13.31, 'Fi0':54 , 'Hs':3.5, 'Occ': 3.152409 },
    {'T': 13.31, 'Fi0':54 , 'Hs':4.5, 'Occ': 0.012711 },
    {'T': 13.31, 'Fi0':90 , 'Hs':0.5, 'Occ': 0.038134 },
    {'T': 13.31, 'Fi0':90 , 'Hs':1.5, 'Occ': 18.73649 },
    {'T': 13.31, 'Fi0':90 , 'Hs':2.5, 'Occ': 28.01999 },
    {'T': 13.31, 'Fi0':90 , 'Hs':3.5, 'Occ': 3.152409 },
    {'T': 13.31, 'Fi0':90 , 'Hs':4.5, 'Occ': 0.012711 },

    {'T': 15.83, 'Fi0':-18  , 'Hs':0.5, 'Occ': 0.016948},
    {'T': 15.83, 'Fi0':-18 , 'Hs':1.5, 'Occ': 6.321766},
    {'T': 15.83, 'Fi0':-18 , 'Hs':2.5, 'Occ': 14.054489},
    {'T': 15.83, 'Fi0':-18 , 'Hs':3.5, 'Occ': 4.122707},
    {'T': 15.83, 'Fi0':-18 , 'Hs':4.5, 'Occ': 0.385577},
    {'T': 15.83, 'Fi0':18  , 'Hs':0.5, 'Occ': 0.016948},
    {'T': 15.83, 'Fi0':18  , 'Hs':1.5, 'Occ': 6.321766},
    {'T': 15.83, 'Fi0':18  , 'Hs':2.5, 'Occ': 14.05448},
    {'T': 15.83, 'Fi0':18  , 'Hs':3.5, 'Occ': 4.122707},
    {'T': 15.83, 'Fi0':18  , 'Hs':4.5, 'Occ': 0.385577},
    {'T': 15.83, 'Fi0':54 , 'Hs':0.5, 'Occ': 0.016948 },
    {'T': 15.83, 'Fi0':54 , 'Hs':1.5, 'Occ': 6.321766 },
    {'T': 15.83, 'Fi0':54 , 'Hs':2.5, 'Occ': 14.05448 },
    {'T': 15.83, 'Fi0':54 , 'Hs':3.5, 'Occ': 4.122707 },
    {'T': 15.83, 'Fi0':54 , 'Hs':4.5, 'Occ': 0.385577 },
    {'T': 15.83, 'Fi0':90 , 'Hs':0.5, 'Occ': 0.016948 },
    {'T': 15.83, 'Fi0':90 , 'Hs':1.5, 'Occ': 6.321766 },
    {'T': 15.83, 'Fi0':90 , 'Hs':2.5, 'Occ': 14.05448 },
    {'T': 15.83, 'Fi0':90 , 'Hs':3.5, 'Occ': 4.122707 },
    {'T': 15.83,'Fi0':90 , 'Hs':4.5, 'Occ': 0.385577}
]

# Crear un DataFrame vacío con las columnas 'Fi0', 'Hs' y 'S'
valores_cerc = pd.DataFrame(valores)
valores_cerc.tail()

,T,Fi0,Hs,Occ
47,15.83,90,0.5,0.016948
48,15.83,90,1.5,6.321766
49,15.83,90,2.5,14.054480
50,15.83,90,3.5,4.122707
51,15.83,90,4.5,0.385577


In [14]:
valores_cerc.to_csv("valores_cerc.csv", index=False)